# Türkçe Duygu/Saldırganlık Analizi - Transformer Model Eğitimi
## BERTurk ve ElecTRa-TR Modelleri ile Fine-tuning

Bu notebook'ta Türkçe siber zorbalık tespiti için iki farklı Transformer modeli eğitilecektir:
- **BERTurk**: dbmdz/bert-base-turkish-cased
- **ElecTRa-TR**: dbmdz/electra-base-turkish-cased-discriminator


In [ ]:
# additional_chat_templates hatasını önle
# Slow tokenizer kullanacağız (use_fast=False)
print("✅ Slow tokenizer kullanılacak (additional_chat_templates problemi yok)")


## 1. Gerekli Kütüphanelerin Yüklenmesi


In [ ]:
# Gerekli kütüphaneleri yükle
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    TrainingArguments, Trainer, EarlyStoppingCallback
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
import warnings
warnings.filterwarnings('ignore')

# CUDA kontrolü
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Kullanılan cihaz: {device}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'Yok'}")


## 2. Dataset Sınıfı Tanımlama


In [ ]:
class TurkishSentimentDataset(Dataset):
    """Türkçe duygu analizi veri seti"""
    
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

print("✅ Dataset sınıfı tanımlandı!")


## 3. Yardımcı Fonksiyonlar


In [ ]:
def load_and_preprocess_data(file_path):
    """Veri setini yükle ve ön işle"""
    print("Veri seti yükleniyor...")
    df = pd.read_csv(file_path)
    
    # Veri temizliği
    df = df.dropna()
    df['text'] = df['text'].astype(str)
    df['label'] = pd.to_numeric(df['label'], errors='coerce')
    df = df.dropna()
    
    # Label'ları temizle (sadece 0-4 arası sayıları kabul et)
    df = df[df['label'].isin([0, 1, 2, 3, 4])]
    
    print(f"Temizlenmiş veri seti boyutu: {len(df)}")
    print(f"Label dağılımı:")
    print(df['label'].value_counts().sort_index())
    
    return df

def compute_metrics(eval_pred):
    """Model performans metrikleri"""
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)
    
    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

def train_model(model_name, train_dataset, val_dataset, num_labels=5, output_dir=None):
    """Model eğitimi"""
    print(f"\n{model_name} modeli eğitiliyor...")
    
    # Tokenizer ve model yükle (use_fast=False ile additional_chat_templates hatası önlenir)
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=num_labels,
        problem_type="single_label_classification"
    )
    
    # Model'i GPU'ya taşı
    model.to(device)
    
    # Eğitim parametreleri
    training_args = TrainingArguments(
        output_dir=output_dir or f'./results_{model_name.replace("/", "_")}',
        num_train_epochs=3,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=16,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir=f'./logs_{model_name.replace("/", "_")}',
        logging_steps=100,
        evaluation_strategy="steps",
        eval_steps=500,
        save_strategy="steps",
        save_steps=500,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        greater_is_better=True,
        report_to=None,  # TensorBoard'u kapat
        save_total_limit=2,
        learning_rate=2e-5,
        fp16=torch.cuda.is_available(),  # GPU varsa mixed precision kullan
    )
    
    # Trainer oluştur
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )
    
    # Eğitimi başlat
    print("Eğitim başlıyor...")
    trainer.train()
    
    # En iyi modeli kaydet
    trainer.save_model()
    tokenizer.save_pretrained(output_dir or f'./results_{model_name.replace("/", "_")}')
    
    # Test seti üzerinde değerlendirme
    print("\nModel değerlendiriliyor...")
    eval_results = trainer.evaluate()
    print(f"Değerlendirme sonuçları: {eval_results}")
    
    return trainer, tokenizer, model

print("✅ Fonksiyonlar tanımlandı!")


## 4. Eğitim Veri Setini Yükleme

**Kaggle'da veri setini yüklemek için:**
1. Data -> Add Data -> Upload Data ile CSV dosyasını yükleyin
2. Veya Dataset olarak Kaggle'da yayınlayıp buradan import edin
3. Dosya yolunu aşağıdaki gibi güncelleyin: `/kaggle/input/your-dataset-name/nlpaug_turkish_augmented.csv`


In [ ]:
# Kaggle'da dosya yolu - kendi dataset adınıza göre güncelleyin
# Örnek: file_path = '/kaggle/input/turkish-cyberbullying/nlpaug_turkish_augmented.csv'
file_path = '/kaggle/input/turkish-cyberbullying-dataset/nlpaug_turkish_augmented.csv'

# Veri setini yükle
df = load_and_preprocess_data(file_path)

# Train/validation split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].values, df['label'].values, 
    test_size=0.2, random_state=42, stratify=df['label']
)

print(f"\nEğitim seti: {len(train_texts)} örnek")
print(f"Validation seti: {len(val_texts)} örnek")


## 5. Model Eğitimi

### 5.1 BERTurk Modeli


In [ ]:
print("="*60)
print("MODEL 1: BERTurk")
print("="*60)

model_name_berturk = "dbmdz/bert-base-turkish-cased"

try:
    # Tokenizer yükle (use_fast=False ile additional_chat_templates hatası önlenir)
    tokenizer_berturk = AutoTokenizer.from_pretrained(model_name_berturk, use_fast=False)
    
    # Dataset'leri oluştur
    train_dataset_berturk = TurkishSentimentDataset(train_texts, train_labels, tokenizer_berturk)
    val_dataset_berturk = TurkishSentimentDataset(val_texts, val_labels, tokenizer_berturk)
    
    # Model eğit
    trainer_berturk, tokenizer_berturk, model_berturk = train_model(
        model_name_berturk, train_dataset_berturk, val_dataset_berturk,
        num_labels=5,
        output_dir='./turkish_sentiment_bert-base-turkish-cased'
    )
    
    # Sonuçları kaydet
    results_berturk = trainer_berturk.evaluate()
    
    print("\n✅ BERTurk modeli başarıyla eğitildi!")
    print(f"📊 Sonuçlar: {results_berturk}")
    
except Exception as e:
    print(f"❌ Hata: BERTurk eğitilirken hata oluştu: {str(e)}")


### 5.2 ElecTRa-TR Modeli


In [ ]:
# GPU belleğini temizle
if 'model_berturk' in locals():
    del model_berturk, trainer_berturk
    torch.cuda.empty_cache()

print("="*60)
print("MODEL 2: ElecTRa-TR")
print("="*60)

model_name_electra = "dbmdz/electra-base-turkish-cased-discriminator"

try:
    # Tokenizer yükle (use_fast=False ile additional_chat_templates hatası önlenir)
    tokenizer_electra = AutoTokenizer.from_pretrained(model_name_electra, use_fast=False)
    
    # Dataset'leri oluştur
    train_dataset_electra = TurkishSentimentDataset(train_texts, train_labels, tokenizer_electra)
    val_dataset_electra = TurkishSentimentDataset(val_texts, val_labels, tokenizer_electra)
    
    # Model eğit
    trainer_electra, tokenizer_electra, model_electra = train_model(
        model_name_electra, train_dataset_electra, val_dataset_electra,
        num_labels=5,
        output_dir='./turkish_sentiment_electra-base-turkish-cased-discriminator'
    )
    
    # Sonuçları kaydet
    results_electra = trainer_electra.evaluate()
    
    print("\n✅ ElecTRa-TR modeli başarıyla eğitildi!")
    print(f"📊 Sonuçlar: {results_electra}")
    
except Exception as e:
    print(f"❌ Hata: ElecTRa-TR eğitilirken hata oluştu: {str(e)}")


## 6. Model Karşılaştırması


In [ ]:
# Sonuçları karşılaştır
print("\n" + "="*60)
print("MODEL KARŞILAŞTIRMASI (Validation Set)")
print("="*60)

comparison_df = pd.DataFrame({
    'Model': ['BERTurk', 'ElecTRa-TR'],
    'Accuracy': [results_berturk.get('eval_accuracy', 0), results_electra.get('eval_accuracy', 0)],
    'F1': [results_berturk.get('eval_f1', 0), results_electra.get('eval_f1', 0)],
    'Precision': [results_berturk.get('eval_precision', 0), results_electra.get('eval_precision', 0)],
    'Recall': [results_berturk.get('eval_recall', 0), results_electra.get('eval_recall', 0)]
})

print(comparison_df.to_string(index=False))

# En iyi modeli belirle
best_model_idx = comparison_df['F1'].idxmax()
print(f"\n🏆 En iyi model: {comparison_df.iloc[best_model_idx]['Model']} (F1: {comparison_df.iloc[best_model_idx]['F1']:.4f})")


## 7. Test Seti Yükleme ve Hazırlık

**Not:** Test setinizi (`test_set_siber_zorbalik_v2.csv`) Kaggle'a yükleyin ve yolu güncelleyin.


In [ ]:
# Test veri setini yükle
# Kaggle'da dosya yolu - kendi dataset adınıza göre güncelleyin
test_file_path = '/kaggle/input/turkish-cyberbullying-dataset/test_set_siber_zorbalik_v2.csv'

print("📂 Test veri seti yükleniyor...")
print("="*60)

# CSV'yi oku
df_test = pd.read_csv(test_file_path)

print(f"📊 Orijinal test seti boyutu: {df_test.shape}")

# Veri temizliği
df_test['label'] = pd.to_numeric(df_test['label'], errors='coerce')
df_test = df_test[df_test['label'].isin([0, 1, 2, 3, 4])]
df_test = df_test.dropna(subset=['comment', 'label'])
df_test['label'] = df_test['label'].astype(int)
df_test = df_test[df_test['comment'].str.strip() != '']
df_test = df_test.reset_index(drop=True)

print(f"✅ Temizlenmiş test seti boyutu: {df_test.shape}")
print(f"\n📈 Test seti sınıf dağılımı:")
print(df_test['label'].value_counts().sort_index())

# Test verilerini hazırla
test_texts = df_test['comment'].values
test_labels = df_test['label'].values


## 8. Test Seti Tahminleri

### 8.1 BERTurk ile Test Seti Tahmini


### 8.2 ElecTRa-TR ile Test Seti Tahmini


In [ ]:
print("\n" + "="*60)
print("BERTURK - TEST SETİ TAHMİNLERİ")
print("="*60)

# Sınıf isimleri
class_names = [
    'No Harassment / Neutral',
    'Direct Insult / Profanity',
    'Sexist / Sexual Implication',
    'Sarcasm / Microaggression',
    'Appearance-based Criticism'
]

# Model'i eval moduna al
model_berturk.eval()
model_berturk.to(device)

# Test dataset oluştur
test_dataset_berturk = TurkishSentimentDataset(test_texts, test_labels, tokenizer_berturk)

# Tahminleri al
predictions_berturk = trainer_berturk.predict(test_dataset_berturk)
y_pred_berturk = np.argmax(predictions_berturk.predictions, axis=1)

# Accuracy hesapla
accuracy_berturk = accuracy_score(test_labels, y_pred_berturk)

print(f"\n📊 Test Accuracy: {accuracy_berturk:.4f}")
print(f"\n📋 Classification Report:\n")
print(classification_report(test_labels, y_pred_berturk, target_names=class_names))

# Her örnek için detaylı sonuç
print("\n" + "="*60)
print("DETAYLI TAHMİN SONUÇLARI")
print("="*60)

results_list_berturk = []
for i in range(len(test_texts)):
    result = {
        'Metin': test_texts[i],
        'Gerçek': class_names[test_labels[i]],
        'Tahmin': class_names[y_pred_berturk[i]],
        'Doğru': '✓' if test_labels[i] == y_pred_berturk[i] else '✗'
    }
    results_list_berturk.append(result)
    print(f"\n{i+1}. {result['Metin'][:50]}...")
    print(f"   Gerçek: {result['Gerçek']}")
    print(f"   Tahmin: {result['Tahmin']} {result['Doğru']}")

# DataFrame olarak kaydet
df_results_berturk = pd.DataFrame(results_list_berturk)
df_results_berturk.to_csv('berturk_test_predictions.csv', index=False, encoding='utf-8-sig')
print("\n💾 BERTurk tahminleri 'berturk_test_predictions.csv' dosyasına kaydedildi!")


In [ ]:
print("\n" + "="*60)
print("ELECTRA-TR - TEST SETİ TAHMİNLERİ")
print("="*60)

# Model'i eval moduna al
model_electra.eval()
model_electra.to(device)

# Test dataset oluştur
test_dataset_electra = TurkishSentimentDataset(test_texts, test_labels, tokenizer_electra)

# Tahminleri al
predictions_electra = trainer_electra.predict(test_dataset_electra)
y_pred_electra = np.argmax(predictions_electra.predictions, axis=1)

# Accuracy hesapla
accuracy_electra = accuracy_score(test_labels, y_pred_electra)

print(f"\n📊 Test Accuracy: {accuracy_electra:.4f}")
print(f"\n📋 Classification Report:\n")
print(classification_report(test_labels, y_pred_electra, target_names=class_names))

# Her örnek için detaylı sonuç
print("\n" + "="*60)
print("DETAYLI TAHMİN SONUÇLARI")
print("="*60)

results_list_electra = []
for i in range(len(test_texts)):
    result = {
        'Metin': test_texts[i],
        'Gerçek': class_names[test_labels[i]],
        'Tahmin': class_names[y_pred_electra[i]],
        'Doğru': '✓' if test_labels[i] == y_pred_electra[i] else '✗'
    }
    results_list_electra.append(result)
    print(f"\n{i+1}. {result['Metin'][:50]}...")
    print(f"   Gerçek: {result['Gerçek']}")
    print(f"   Tahmin: {result['Tahmin']} {result['Doğru']}")

# DataFrame olarak kaydet
df_results_electra = pd.DataFrame(results_list_electra)
df_results_electra.to_csv('electra_test_predictions.csv', index=False, encoding='utf-8-sig')
print("\n💾 ElecTRa-TR tahminleri 'electra_test_predictions.csv' dosyasına kaydedildi!")


## 9. Test Seti Sonuçlarının Karşılaştırılması


In [ ]:
print("\n" + "="*60)
print("TEST SETİ - MODEL KARŞILAŞTIRMASI")
print("="*60)

test_comparison_df = pd.DataFrame({
    'Model': ['BERTurk', 'ElecTRa-TR'],
    'Test Accuracy': [accuracy_berturk, accuracy_electra],
    'Doğru Tahmin': [
        (y_pred_berturk == test_labels).sum(),
        (y_pred_electra == test_labels).sum()
    ],
    'Yanlış Tahmin': [
        (y_pred_berturk != test_labels).sum(),
        (y_pred_electra != test_labels).sum()
    ],
    'Toplam': [len(test_labels), len(test_labels)]
})

print("\n" + test_comparison_df.to_string(index=False))

# En iyi test performansı
best_test_idx = test_comparison_df['Test Accuracy'].idxmax()
print(f"\n🏆 Test setinde en iyi model: {test_comparison_df.iloc[best_test_idx]['Model']}")
print(f"   Accuracy: {test_comparison_df.iloc[best_test_idx]['Test Accuracy']:.4f}")

# Karşılaştırmalı sonuçları kaydet
test_comparison_df.to_csv('test_set_comparison.csv', index=False)
print("\n💾 Karşılaştırma sonuçları 'test_set_comparison.csv' dosyasına kaydedildi!")


## 10. Özet ve Kaydedilen Dosyalar


In [ ]:
print("\n" + "="*60)
print("📊 PROJE ÖZETİ")
print("="*60)

print(f"\n✅ Toplam eğitim verisi: {len(df)}")
print(f"✅ Training set: {len(train_texts)} örnek")
print(f"✅ Validation set: {len(val_texts)} örnek")
print(f"✅ Test set: {len(test_texts)} örnek")

print(f"\n✅ Eğitilen modeller:")
print(f"   1. BERTurk (dbmdz/bert-base-turkish-cased)")
print(f"      - Validation F1: {results_berturk.get('eval_f1', 0):.4f}")
print(f"      - Test Accuracy: {accuracy_berturk:.4f}")
print(f"   2. ElecTRa-TR (dbmdz/electra-base-turkish-cased-discriminator)")
print(f"      - Validation F1: {results_electra.get('eval_f1', 0):.4f}")
print(f"      - Test Accuracy: {accuracy_electra:.4f}")

print(f"\n✅ Kaydedilen model dizinleri:")
print("   - ./turkish_sentiment_bert-base-turkish-cased/")
print("   - ./turkish_sentiment_electra-base-turkish-cased-discriminator/")

print(f"\n✅ Kaydedilen tahmin dosyaları:")
print("   - berturk_test_predictions.csv")
print("   - electra_test_predictions.csv")
print("   - test_set_comparison.csv")

print("\n" + "="*60)
print("🎉 TÜM İŞLEMLER BAŞARIYLA TAMAMLANDI!")
print("="*60)
